In [1]:
import pandas as pd


from sklearn.preprocessing import OneHotEncoder




# Preprocessing ev_stations data

In [2]:
stations_df = pd.read_csv(r'C:\Users\clava\data_miners\resources\TRNSFRM_ev_stations_v1.csv')
stations_df

,Station_Name,Street_Address,City,State_Code,Zip_Code,Status_Code,Access_Days_Time,EV_Level1_EVSE_Num,EV_Level2_EVSE_Num,EV_DC_Fast_Count,EV_Network,Latitude,Longitude,EV_ID,Owner_Type_Code,Open_Date,Access_Code,Facility_Type,Charging_Cost
0,LADWP - Truesdale Center,11797 Truesdale St,Sun Valley,CA,91352,E,Fleet use only,NaN,39.0,3.0,Non-Networked,34.248319,-118.387971,1517,LG,10/15/1999,private,UTILITY,NaN
1,LADWP - West LA District Office,1394 S Sepulveda Blvd,Los Angeles,CA,90024,E,NaN,NaN,4.0,NaN,Non-Networked,34.052542,-118.448504,1519,LG,2/28/2020,private,UTILITY,Free
2,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,E,24 hours daily; pay lot,NaN,12.0,NaN,Non-Networked,34.040539,-118.271387,1523,P,8/30/1995,public,PARKING_GARAGE,NaN
3,LADWP - John Ferraro Building,111 N Hope St,Los Angeles,CA,90012,E,For fleet and employee use only,NaN,311.0,2.0,Non-Networked,34.059133,-118.248589,1525,LG,10/15/1999,private,UTILITY,NaN
4,LADWP - Haynes Power Plant,6801 E 2nd St,Long Beach,CA,90803,E,Fleet use only,NaN,19.0,1.0,Non-Networked,33.759802,-118.096665,1531,LG,5/1/2018,private,UTILITY,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50216,VIP LOT STATION1,1501 Kirkwood Meadows Dr,Markleeville,CA,96120,E,24 hours daily,NaN,2.0,NaN,ChargePoint Network,38.684660,-120.065169,204783,NaN,1/14/2022,public,NaN,NaN
50217,Prunedale Shopping Center,8065 San Miguel Canyon Rd,Salinas,CA,93907,E,24 hours daily,NaN,NaN,6.0,eVgo Network,36.801716,-121.664153,204784,NaN,1/14/2022,public,NaN,NaN
50218,Beaverton Electric Avenue,11665 SW Beaverton Hillsdale Hwy,Beaverton,OR,97005,E,24 hours daily,NaN,NaN,2.0,Greenlots,45.489030,-122.798151,204785,NaN,1/14/2022,public,NaN,NaN
50219,Shell - Inman,2040 Highway 292,Inman,SC,29349,E,24 hours daily,NaN,NaN,1.0,Greenlots,35.082476,-82.058433,204786,NaN,1/14/2022,public,NaN,NaN


In [3]:
stations_df.dtypes

Station_Name           object
Street_Address         object
City                   object
State_Code             object
Zip_Code               object
Status_Code            object
Access_Days_Time       object
EV_Level1_EVSE_Num    float64
EV_Level2_EVSE_Num    float64
EV_DC_Fast_Count      float64
EV_Network             object
Latitude              float64
Longitude             float64
EV_ID                   int64
Owner_Type_Code        object
Open_Date              object
Access_Code            object
Facility_Type          object
Charging_Cost          object
dtype: object

In [4]:
stations_df.Zip_Code.nunique()

9465

In [5]:
# Drop columns with unusable date
stations_df = stations_df.drop(columns=['Status_Code', 'Access_Days_Time', 'EV_ID', 'Owner_Type_Code', 'Open_Date', 'Access_Code', 'Facility_Type', 'Charging_Cost'])

# bin network type
other_network = ['Volta', 'EV Connect', 'OpConnect', 'SemaCharge Network', 'Tesla Destination',
                 'Greenlots', 'EVGATEWAY', 'POWERFLEX', 'eVgo Network', 'Webasto', 'CHARGELAB',
                 'AMPUP', 'EVCS', 'Blink Network', 'FCN', 'Tesla', 'Electrify America', 'FLO',
                 'FPLEV', 'ZEFNET', 'LIVINGSTON', 'Electrify Canada']

for network in other_network:
  stations_df.EV_Network = stations_df.EV_Network.replace(network, 'Other-Network')

stations_df.EV_Network = stations_df.EV_Network.replace('ChargePoint Network', 'ChargePoint')

# drop rows with States not used in current model
states = ['MI', 'MN', 'NJ', 'NY', 'OR', 'TX', 'WA', 'WI']

stations_df = stations_df[stations_df.State_Code.isin(states) == True]

# FUTURE OPTION - Classify Charging_Cost as FREE or PAID - Need solution to classify NaNs
# stations_df['Charging_Cost'] = stations_df['Charging_Cost'].replace(to_replace = ('Free', 'Paid'), value = ('FREE', 'PAID'))

# replace NaNs in EVSE type columns
stations_df['EV_Level1_EVSE_Num'] = stations_df['EV_Level1_EVSE_Num'].fillna(0)
stations_df['EV_Level2_EVSE_Num'] = stations_df['EV_Level2_EVSE_Num'].fillna(0)
stations_df['EV_DC_Fast_Count'] = stations_df['EV_DC_Fast_Count'].fillna(0)

# reorder columns in table
stations_df = stations_df[['Station_Name', 'Street_Address', 'City', 'State_Code', 'Zip_Code', 'Latitude', 'Longitude', 'EV_Level1_EVSE_Num', 'EV_Level2_EVSE_Num', 'EV_DC_Fast_Count', 'EV_Network']]

# rename column names
stations_df = stations_df.rename(columns={
    'Zip_Code': 'ZIP_CODE',
    'EV_Level1_EVSE_Num': 'EV_LEVEL_1',
    'EV_Level2_EVSE_Num': 'EV_LEVEL_2',
    'EV_DC_Fast_Count': 'EV_FAST',
    'EV_Network': 'NETWORK_TYPE'})

# drop rows with nan values
stations_df = stations_df.dropna().reset_index(drop=True)

# OneHotEncode Network Type
enc = OneHotEncoder(sparse=False)
encode_df = pd.DataFrame(enc.fit_transform(stations_df.NETWORK_TYPE.values.reshape(-1,1)))
encode_df.columns = enc.get_feature_names_out(['NETWORK_TYPE'])

stations_df = stations_df.merge(encode_df, left_index=True, right_index=True).drop('NETWORK_TYPE', 1)

stations_df.head()

C:\Users\clava\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


,Station_Name,Street_Address,City,State_Code,ZIP_CODE,Latitude,Longitude,EV_LEVEL_1,EV_LEVEL_2,EV_FAST,NETWORK_TYPE_ChargePoint,NETWORK_TYPE_Non-Networked,NETWORK_TYPE_Other-Network
0,City of Lacey - City Hall Parking,420 College St,Lacey,WA,98503,47.044011,-122.822404,0.0,4.0,0.0,0.0,1.0,0.0
1,Oregon Museum of Science and Industry (OMSI) -...,1945 SE Water Ave,Portland,OR,97214,45.508396,-122.664667,2.0,2.0,0.0,0.0,1.0,0.0
2,Seattle-Tacoma International Airport - General...,17801 Pacific Hwy S,Seattle,WA,98188,47.443377,-122.296229,36.0,0.0,0.0,0.0,1.0,0.0
3,Travis Park United Methodist Church,230 E Travis St,San Antonio,TX,78205,29.427254,-98.490136,2.0,0.0,0.0,0.0,1.0,0.0
4,City of Keizer - City Hall,930 Chemawa Rd NE,Salem,OR,97303,44.995740,-123.019758,2.0,0.0,0.0,0.0,1.0,0.0


# Preprocessing Registration Data

In [6]:
# read in CSV files
mi = (r'C:\Users\clava\data_miners\resources\mi_ev_registrations_public.csv')
mn = (r'C:\Users\clava\data_miners\resources\mn_ev_registrations_public.csv')
nj = (r'C:\Users\clava\data_miners\resources\nj_ev_registrations_public.csv')
ny = (r'C:\Users\clava\data_miners\resources\ny_ev_registrations_public.csv')
ore = (r'C:\Users\clava\data_miners\resources\or_ev_registrations_public.csv')
tx = (r'C:\Users\clava\data_miners\resources\tx_ev_registrations_public.csv')
wa = (r'C:\Users\clava\data_miners\resources\wa_ev_registrations_public.csv')
wi = (r'C:\Users\clava\data_miners\resources\wi_ev_registrations_public.csv')

# function to create dataframes from csvs
def clean_state(state):
  state_df = pd.read_csv(state)

  #state_df = state_df.groupby(['Zip_code','State_Code'], as_index = False).agg({'DMV_ID':'count'})
  state_df = state_df.Zip_code.value_counts().rename_axis('ZIP_CODE').reset_index(name='REG_COUNTS')
    
  state_df['ZIP_CODE'] = state_df['ZIP_CODE'].astype(int)

  return state_df

# create dataframes using clean_state function
mi_df = clean_state(mi)
mn_df = clean_state(mn)
nj_df = clean_state(nj)
ny_df = clean_state(ny)
ore_df = clean_state(ore)
tx_df = clean_state(tx)
wa_df = clean_state(wa)
wi_df = clean_state(wi)





In [7]:
# ny_df = ny_df.loc[(ny_df['ZIP_CODE'] >= 10001) & (ny_df['ZIP_CODE'] <= 14975)]
# mi_df = mi_df.loc[(mi_df['ZIP_CODE'] >= 48001) & (mi_df['ZIP_CODE'] <= 49971)]
# mn_df = mn_df.loc[(mn_df['ZIP_CODE'] >= 55001) & (mn_df['ZIP_CODE'] <= 56763)]
# nj_df = nj_df.loc[(nj_df['ZIP_CODE'] >= 7001) & (nj_df['ZIP_CODE'] <= 8989)]
# ore_df = ore_df.loc[(ore_df['ZIP_CODE'] >= 97001) & (ore_df['ZIP_CODE'] <= 979201)]
# #tx_df = tx_df.loc[(tx_df['ZIP_CODE'] >= 10001) & (tx_df['ZIP_CODE'] <= 14975)]
# wi_df = wi_df.loc[(wi_df['ZIP_CODE'] >= 53001) & (wi_df['ZIP_CODE'] <= 54990)]
# wa_df = wa_df.loc[(wa_df['ZIP_CODE'] >= 98001) & (wa_df['ZIP_CODE'] <= 99403)]

In [8]:
# create registration_df and set index as ZIP_CODE
registration_df = pd.concat([mi_df,
                             mn_df,
                             nj_df,
                             ny_df,
                             ore_df,
                             tx_df,
                             wa_df,
                             wi_df])



In [9]:
registration_df

,ZIP_CODE,REG_COUNTS
0,48033,2048
1,48103,1252
2,48105,698
3,48104,545
4,48009,519
...,...,...
909,28036,1
910,76014,1
911,54871,1
912,54538,1


In [12]:
#registration_df.DMV_ID.sum()

In [13]:
#registration_df = registration_df.rename(columns={'Zip_code':'ZIP_CODE', 'State_Code':'State_Code','DMV_ID':'REG_COUNTS'})

In [14]:
registration_df.ZIP_CODE.nunique()

8350

In [15]:
#tx_df['REG_COUNTS'].sum()

In [16]:
#ny_df['REG_COUNTS'].sum()

In [17]:
registration_df['REG_COUNTS'].sum()

3075068

In [18]:
registration_df

,ZIP_CODE,REG_COUNTS
0,48033,2048
1,48103,1252
2,48105,698
3,48104,545
4,48009,519
...,...,...
909,28036,1
910,76014,1
911,54871,1
912,54538,1


In [19]:
ny_registrations = registration_df.loc[(registration_df['ZIP_CODE'] >= 10001) & (registration_df['ZIP_CODE'] <= 14975)]
ny_registrations.REG_COUNTS.sum()

1043914

In [20]:
no_registrations = registration_df.loc[(registration_df['ZIP_CODE'] >= 0) & (registration_df['ZIP_CODE'] <= 7000)]
no_registrations.REG_COUNTS.sum()

1071

In [21]:
# count each level type by zip code
#registration_df = registration_df.groupby(['ZIP_CODE'], as_index=False).sum()
#registration_df

In [22]:
registration_df['REG_COUNTS'] = registration_df['REG_COUNTS'].astype(int)
registration_df['ZIP_CODE'] = registration_df['ZIP_CODE'].astype(int)

# Preprocessing Median Income Data

In [23]:
median_inc = (r'C:\Users\clava\data_miners\resources\TRNSFRM_census_median_income.csv')
median_df = pd.read_csv(median_inc)
median_df = median_df[median_df.MEDIAN_INCOME != '2,500-']
median_df = median_df[median_df.MEDIAN_INCOME != '250,000+']
median_df = median_df[median_df.MEDIAN_INCOME != '-']
median_df['TOTAL_HOUSEHOLDS'] = median_df['TOTAL_HOUSEHOLDS'].astype(int)
median_df['MEDIAN_INCOME'] = median_df['MEDIAN_INCOME'].astype(int)
median_df.dtypes

ZIP_CODE            int64
TOTAL_HOUSEHOLDS    int32
MEDIAN_INCOME       int32
dtype: object

In [24]:
median_df

,ZIP_CODE,TOTAL_HOUSEHOLDS,MEDIAN_INCOME
0,601,5509,14361
1,602,12740,16807
2,603,19228,16049
3,606,1946,12119
4,610,8795,19898
...,...,...,...
33113,99921,814,52188
33114,99922,117,33750
33116,99925,317,62917
33117,99926,487,56188


In [25]:
median_df.nunique()

ZIP_CODE            30797
TOTAL_HOUSEHOLDS    10650
MEDIAN_INCOME       19970
dtype: int64

In [26]:
registration_df.REG_COUNTS.sum()

3075068

# Final model Dataframe

In [27]:
# Drop location info for ml model
model_df = stations_df.drop(columns=['Station_Name',
                                     'Street_Address',
                                     'City',
                                     #'State_Code',
                                     'Latitude',
                                     'Longitude'])



# change data types for sorting and merging
model_df['ZIP_CODE'] = model_df['ZIP_CODE'].astype(int)
model_df['EV_LEVEL_1'] = model_df['EV_LEVEL_1'].astype(int)
model_df['EV_LEVEL_2'] = model_df['EV_LEVEL_2'].astype(int)
model_df['EV_FAST'] = model_df['EV_FAST'].astype(int)
model_df['NETWORK_TYPE_ChargePoint'] = model_df['NETWORK_TYPE_ChargePoint'].astype(int)
model_df['NETWORK_TYPE_Non-Networked'] = model_df['NETWORK_TYPE_Non-Networked'].astype(int)
model_df['NETWORK_TYPE_Other-Network'] = model_df['NETWORK_TYPE_Other-Network'].astype(int)





model_df

,State_Code,ZIP_CODE,EV_LEVEL_1,EV_LEVEL_2,EV_FAST,NETWORK_TYPE_ChargePoint,NETWORK_TYPE_Non-Networked,NETWORK_TYPE_Other-Network
0,WA,98503,0,4,0,0,1,0
1,OR,97214,2,2,0,0,1,0
2,WA,98188,36,0,0,0,1,0
3,TX,78205,2,0,0,0,1,0
4,OR,97303,2,0,0,0,1,0
...,...,...,...,...,...,...,...,...
10667,NY,12019,0,2,0,1,0,0
10668,NY,12019,0,2,0,1,0,0
10669,TX,77868,0,1,0,1,0,0
10670,TX,77868,0,1,0,1,0,0


In [28]:
model_df = stations_df.groupby(['ZIP_CODE','State_Code'], as_index=False).sum()
model_df

,ZIP_CODE,State_Code,Latitude,Longitude,EV_LEVEL_1,EV_LEVEL_2,EV_FAST,NETWORK_TYPE_ChargePoint,NETWORK_TYPE_Non-Networked,NETWORK_TYPE_Other-Network
0,10001,NY,448.248742,-813.937811,0.0,38.0,0.0,0.0,1.0,10.0
1,10002,NY,244.304916,-443.938841,0.0,15.0,4.0,0.0,0.0,6.0
2,10003,NY,651.718882,-1183.851951,0.0,52.0,0.0,0.0,0.0,16.0
3,10004,NY,40.704976,-74.009682,0.0,3.0,0.0,0.0,0.0,1.0
4,10005,NY,122.116193,-222.026384,0.0,7.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...
2589,99354,WA,2131.832985,-5486.839772,36.0,48.0,0.0,0.0,45.0,1.0
2590,99361,WA,46.270749,-118.153251,0.0,1.0,0.0,0.0,0.0,1.0
2591,99362,WA,736.861346,-1893.326012,1.0,27.0,0.0,1.0,3.0,12.0
2592,99403,WA,46.425498,-117.048650,0.0,2.0,0.0,0.0,0.0,1.0


In [29]:
model_df.dtypes

ZIP_CODE                       object
State_Code                     object
Latitude                      float64
Longitude                     float64
EV_LEVEL_1                    float64
EV_LEVEL_2                    float64
EV_FAST                       float64
NETWORK_TYPE_ChargePoint      float64
NETWORK_TYPE_Non-Networked    float64
NETWORK_TYPE_Other-Network    float64
dtype: object

In [30]:
model_df['ZIP_CODE'] = model_df['ZIP_CODE'].astype(int)

In [31]:
# merge the data fram with median_df
model_df = model_df.merge(median_df, how='left', on='ZIP_CODE')
model_df

,ZIP_CODE,State_Code,Latitude,Longitude,EV_LEVEL_1,EV_LEVEL_2,EV_FAST,NETWORK_TYPE_ChargePoint,NETWORK_TYPE_Non-Networked,NETWORK_TYPE_Other-Network,TOTAL_HOUSEHOLDS,MEDIAN_INCOME
0,10001,NY,448.248742,-813.937811,0.0,38.0,0.0,0.0,1.0,10.0,12997.0,92840.0
1,10002,NY,244.304916,-443.938841,0.0,15.0,4.0,0.0,0.0,6.0,33459.0,36982.0
2,10003,NY,651.718882,-1183.851951,0.0,52.0,0.0,0.0,0.0,16.0,26043.0,118161.0
3,10004,NY,40.704976,-74.009682,0.0,3.0,0.0,0.0,0.0,1.0,1777.0,190223.0
4,10005,NY,122.116193,-222.026384,0.0,7.0,0.0,0.0,0.0,3.0,4363.0,189702.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2589,99354,WA,2131.832985,-5486.839772,36.0,48.0,0.0,0.0,45.0,1.0,9175.0,73369.0
2590,99361,WA,46.270749,-118.153251,0.0,1.0,0.0,0.0,0.0,1.0,750.0,69118.0
2591,99362,WA,736.861346,-1893.326012,1.0,27.0,0.0,1.0,3.0,12.0,15717.0,56665.0
2592,99403,WA,46.425498,-117.048650,0.0,2.0,0.0,0.0,0.0,1.0,8338.0,53079.0


In [32]:
registration_df.dtypes

ZIP_CODE      int32
REG_COUNTS    int32
dtype: object

In [33]:
registration_df.REG_COUNTS.sum()

3075068

In [34]:
registration_df.ZIP_CODE.nunique()

8350

In [35]:
model_df.ZIP_CODE.nunique()

2593

In [36]:
# merge the data frame with registration_df
model_df = model_df.merge(registration_df, how='outer', on='ZIP_CODE')

# fill nans with zero
model_df['REG_COUNTS'] = model_df['REG_COUNTS'].fillna(0)

#model_df = model_df.dropna(how='all')

# change data type for sorting and merging
model_df['REG_COUNTS'] = model_df['REG_COUNTS'].astype(int)

model_df

,ZIP_CODE,State_Code,Latitude,Longitude,EV_LEVEL_1,EV_LEVEL_2,EV_FAST,NETWORK_TYPE_ChargePoint,NETWORK_TYPE_Non-Networked,NETWORK_TYPE_Other-Network,TOTAL_HOUSEHOLDS,MEDIAN_INCOME,REG_COUNTS
0,10001,NY,448.248742,-813.937811,0.0,38.0,0.0,0.0,1.0,10.0,12997.0,92840.0,1
1,10001,NY,448.248742,-813.937811,0.0,38.0,0.0,0.0,1.0,10.0,12997.0,92840.0,9
2,10001,NY,448.248742,-813.937811,0.0,38.0,0.0,0.0,1.0,10.0,12997.0,92840.0,1127
3,10001,NY,448.248742,-813.937811,0.0,38.0,0.0,0.0,1.0,10.0,12997.0,92840.0,4
4,10002,NY,244.304916,-443.938841,0.0,15.0,4.0,0.0,0.0,6.0,33459.0,36982.0,1161
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9351,54120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9352,80015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9353,53082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9354,28036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [37]:
model_df.nunique()


ZIP_CODE                      8433
State_Code                       8
Latitude                      2593
Longitude                     2593
EV_LEVEL_1                      20
EV_LEVEL_2                      83
EV_FAST                         30
NETWORK_TYPE_ChargePoint        41
NETWORK_TYPE_Non-Networked      14
NETWORK_TYPE_Other-Network      25
TOTAL_HOUSEHOLDS              2334
MEDIAN_INCOME                 2441
REG_COUNTS                    1500
dtype: int64

In [38]:
# soft dataframe by zip code
model_df = model_df.sort_values(by='ZIP_CODE')

# change zip code to string and reformat
model_df['ZIP_CODE'] = model_df['ZIP_CODE'].apply(lambda x: '0' + str(x) if x < 10000 else str(x)) 

# set ZIP_CODE as index
model_df = model_df.set_index('ZIP_CODE')

model_df = model_df.dropna(axis=0, how='any')

In [39]:
model_df.dtypes

State_Code                     object
Latitude                      float64
Longitude                     float64
EV_LEVEL_1                    float64
EV_LEVEL_2                    float64
EV_FAST                       float64
NETWORK_TYPE_ChargePoint      float64
NETWORK_TYPE_Non-Networked    float64
NETWORK_TYPE_Other-Network    float64
TOTAL_HOUSEHOLDS              float64
MEDIAN_INCOME                 float64
REG_COUNTS                      int32
dtype: object

In [40]:
model_df.REG_COUNTS.sum()

2403009

In [41]:
model_df.State_Code.isnull().count()

2939

In [42]:
# for state in model_df:
#     if (model_df['State_Code_x'] == ''''):
#         model_df['State_Code_x'].value = model_df['State_Code_y'].value
#     else:
#         model_df['State_Code_x'] = ''
    
#model_df['State_Code_x'] = model_df["State_Code_x"].fillna(model_df["State_Code_y"],axis=0)


In [43]:
model_df

,State_Code,Latitude,Longitude,EV_LEVEL_1,EV_LEVEL_2,EV_FAST,NETWORK_TYPE_ChargePoint,NETWORK_TYPE_Non-Networked,NETWORK_TYPE_Other-Network,TOTAL_HOUSEHOLDS,MEDIAN_INCOME,REG_COUNTS
ZIP_CODE,,,,,,,,,,,,
03266,WI,43.045323,-88.041306,0.0,2.0,0.0,0.0,1.0,0.0,851.0,54904.0,0
07001,NJ,40.573827,-74.290584,0.0,1.0,0.0,0.0,1.0,0.0,5022.0,69849.0,173
07003,NJ,326.418030,-593.539921,0.0,12.0,5.0,4.0,1.0,3.0,18577.0,78034.0,5
07003,NJ,326.418030,-593.539921,0.0,12.0,5.0,4.0,1.0,3.0,18577.0,78034.0,795
07004,NJ,122.643406,-222.838226,0.0,6.0,3.0,0.0,1.0,2.0,2621.0,122063.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
99352,WA,277.592858,-715.725081,0.0,13.0,3.0,1.0,1.0,4.0,12877.0,81410.0,3411
99354,WA,2131.832985,-5486.839772,36.0,48.0,0.0,0.0,45.0,1.0,9175.0,73369.0,2082
99361,WA,46.270749,-118.153251,0.0,1.0,0.0,0.0,0.0,1.0,750.0,69118.0,100


In [44]:
# export model_df for the ML Model predictions.
#model_df.to_csv('viz_ml_df.csv')

In [45]:
# export model_df for the ML Model predictions.
model_df.to_csv('viz_df.csv')